# Una muy breve introducción a los Algoritmos Genéticos en Python


## Repo y Redes

Repo
* https://github.com/bt0DotNinja/charla_pymx

Telegram
* https://t.me/pythonCDMX
* https://t.me/fedoramexico


Twitter
* https://twitter.com/__PyMX__
* https://twitter.com/fedoramexico

## Agenda

* Acerca de mi
* Optimización
* Teoría
  * Algoritmo Básico
* Aprendiendo con el ejemplo: Problema de las $N$ Reinas
  * Representación de las soluciones.
  * Generación de población inicial.
  * Aptitud.
  * Cruza.
  * Mutación
  * Selección.
  * Parámetros del AG
  * Elitismo
  * Ejemplo
* Biblioteca geneticalgorithm2
  * Ejemplo numérico con geneticalgorithm2
* Preguntas finales y saludos sonideros. 


## Acerca de mi

Soy un feliz colaborador del Proyecto Fedora, actualmente me desempeño como ingeniero de redes.

![Fedora Loves Python](./images/fedoralovespython.svg)

Soy egresado de la Universidad Autónoma Metropolitana donde conseguí los grados de Lic. en Ingeniería en Computación (Así dice el titulo) y Maestro en Optimización (Me especialice en algoritmos evolutivos multiobjetivo) [skills](./images/skills.jpeg).

![Ciencia](./images/science.svg)

Y estoy muy contento de estar aquí con la comunidad de Python México.




## Optimización

Es básicamente el conjunto de técnicas para resolver el siguiente problema general**:

$$\min F({x})$$
S.A:
 
$$g_i({x}) \leq 0, i=1,2,\dots,m$$
$$h_i({x}) = 0, i=1,2,\dots,p$$
 
Buscamos el vector ${x}=[x_1,x_2,\dots,x_k]^T$ que optimice la función $F$

![minimos y maximos](./images/extremos.svg)

## Teoría de AG

* Desarrollados por John H. Holland a principios de los 1960s, motivado por resolver problemas de aprendizaje de máquina.
* El algoritmo genético enfatiza la importancia de la cruza sexual (el operador mas divertido como operador principal) sobre el de la mutación (operador secundario), y usa selección probabilística.
* Componentes básicos
  * Representación de las soluciones potenciales del problema
  * Método de creación de una población inicial de posibles soluciones (normalmente un proceso aleatorio)
  * Función de evaluación que juegue el papel del ambiente, clasificando las soluciones en términos de su “aptitud”
  * Operadores genéticos que alteren la composición de los hijos que se producirán para las siguientes generaciones
  * Valores para los diferentes parámetros que utiliza el algoritmo genético (tamaño de la población, probabilidad de cruza, probabilidad de mutación, número máximo de generaciones, etc.)


#### Ventajas
* Trabaja bien con un número alto de variables.
* Realiza búsquedas simultáneas en el espacio de las soluciones.
* Fácil de implementar.


#### Desventajas
* No hay garantía de encontrar el resultado óptimo**.
* Requiere "tunear" los parametros.
* No funciona bien para todas las variedades de algún problema**.

### Algoritmo Básico

```
Generar (aleatoriamente) una población inicial
Ciclar hasta que cierta condición se satisfaga
    Calcular aptitud de cada individuo en la población
    Seleccionar (probabilísticamente) en base a aptitud
    Aplicar operadores genéticos (cruza y mutación) para generar la siguiente población
```

## Aprendiendo con el ejemplo: Problema de las $N$ Reinas

![primero lo primero](./images/teaching.svg)
El problema consistiría en colocar $N$ reinas en un tablero de ajedrez de $N × N$ de tal manera que ninguna de las reinas quede amenazando a otras. Es decir, solo puede haber una reina por fila, columna y diagonal. 

![Como ataca la reina](./images/atacar.JPG)

La variedad clásica de este problema involucra enumerar todas las posibles soluciones en un tablero dado. Para fines didácticos buscaremos sólo una solución usando algoritmos genéticos con $N=20$. La idea es **minimizar** la cantidad de reinas amenazadas en el tablero.

![Minimzar Amenazas](./images/minconflict.gif)


> Nota: El problema de las $N$ reinas con $N = 20$, tiene $39,029,188,884$ soluciones posibles (de estas sólo $4,878,666,808$ son básicas o fundamentales).

In [ ]:
# Bibliotecas y funciones misceláneas

import matplotlib.pyplot as plt
from scipy import optimize
import numpy as np
import random

def dibujar_tablero(sol):
    '''
    Recibe una solución del problema de las n-reinas y la grafica en un tablero de ajedrez
    '''
    chessboard = np.zeros((len(sol),len(sol)))
    chessboard[1::2,0::2] = 1
    chessboard[0::2,1::2] = 1
    plt.imshow(chessboard, cmap='binary')
    for i in range(len(sol)):
        plt.text(i, sol[i], '♕', fontsize=12, ha='center', va='center', color='red')
    plt.show()

def f(X):
    '''
    Funcion de pruebas en el formato scypy optimize
    '''
    x,y = X
    return ( 4 * np.sin(np.pi * x) + 6 * np.sin(np.pi * y)) + (x - 1)**2 + (y - 1)**2

def f2(X):
    '''
    Funcion de pruebas en el formato scypy optimize
    '''
    return ( 4 * np.sin(np.pi * X[0]) + 6 * np.sin(np.pi * X[1])) + (X[0] - 1)**2 + (X[1] - 1)**2
    
def dibujar_estadisticas(stats):
    '''
    Dibuja el progreso del algoritmo genético
    '''
    lGen=list(range(len(stats['max'])))
    
    plt.plot(lGen, stats['max'], lGen, stats['media'], lGen, stats['min'])
    plt.show()

### Representación de las soluciones

Hay distintas formas de representar una solución, como por ejemplo:

* Binaria
* Real**
* Discreta

En el caso de las $N$ reinas, cada tablero (solución), se puede representar como una lista de números enteros, no repetidos, donde la posición en la lista representa la fila en el tablero y el número representa la columna donde se encuentra la reina.

Por ejemplo, la representación del siguiente tablero: 

![tablero de prueba](./images/8queen-solution.jpg)

seria: `[4,0,7,3,1,6,2,5]`

> Iniciamos contando en 0. 

### Generación de población inicial

Usualmente se genera de manera (Pseudo) aleatoria, sin embargo, puede ser generada por otros métodos.

* Debe mantener la representación.
* Usualmente buscamos que este bien distribuida sobre el espacio. 

In [ ]:
def poblacionInicial(tamPob, nR, test=False):
    '''
    Genera una poblacion inicial,
    recibe:
        tamPob: tamaño de población
        nR: Número de reinas
        test: corrida con semilla controlada
    regresa:
        pob: población inicial generada de forma aleatoria.
    '''
    if test:
        random.seed(0)
    ordenada=list(range(nR))
    pob=[]
    for _ in range(tamPob):
        random.shuffle(ordenada)
        pob.append(ordenada[:])
    return pob

### Aptitud

Es la función que indica que tan buena es una solución con respecto a las demas, en nuetro caso tomaremos el número de "amenazas" o conflictos de cada solución. 

In [ ]:
def aptitud(s):
    '''
    Realiza el conteo de conflictos de la entrada
    recibe:
        s: conjunto a evaluar
    regresa:
        n: numero de conflictos
    '''
    n=0
    for e in range(len(s)):
        i=1
        while e+i < len(s):
            if s[e+i] == s[e]+i:
                n+=1
            if s[e+i] == s[e]-i:
                n+=1
            i+=1
            if s[e]+i > len(s) and s[e]-i < 0:
                break
    return n

### Cruza 
Vamos a definir una cruza que no rompa la representación, una opción sencilla es como en el siguiente ejemplo
Dados dos individuos *Padres*:

![cruza](./images/cruza.png)

> Para ser algoritmo genético la cruza debe tomar/combinar dos individuos de la población.

In [ ]:
def cruza(s1,s2, test=False):
    '''
    Realiza la cruza (sexual) de dos individuos
    recibe:
        s1: Padre 1
        s2: Padre 2
    regresa:
        h: hijo
    '''
    nR=len(s1)
    if test:
        random.seed(0)

    a=random.randint(0, nR - 1)
    b=random.randint(0, nR - 1)
    
    while a == b:
        b=random.randint(0, nR - 1)
        
    if a > b:
        a,b = b,a
        
    h=[]
    
    for i in range(nR):
        if s2[i] not in s1[a:b]:
            h.append(s2[i])
    
    return h[:a] + s1[a:b] + h[a:]

### Mutación
La mutación es una operación que permite salir de minimos o maximos locales. Una mutación típica y muy simple consiste en permutar dos elementos del genoma de un individuo modificiacion.

In [ ]:
def mutacion(s,probaMut,test=False):
    '''
    Realiza un intercambio entre dos elementos de indices aleatorios
    recibe:
        s: conjunto respuesta a mutar
        test: corrida con semilla controlada
    regresa:
        r: una permutacion de s, producto del intercambio aleatorio de dos elementos de s.
    '''
    if test:
        random.seed(0)
    if random.random() <= probaMut:
        return s
    a=random.randint(0,len(s) -1)
    b=random.randint(0,len(s) -1)
    #evita que i y j sean iguales
    while a==b:
        b=random.randint(0,len(s) -1)
    S=s[:] #evita mutacion de la lista
    S[a],S[b] = S[b],S[a]
    return S

### Selección
Existen distintas formas de selección, como:
* Adan y Eva.
* Extinción de individuos poco aptos.
* Torneo.
* etc.

Usaremos para este ejemplo una variedad de torneo donde después de la cruza y la mutación, el hijo competirá contra alguno de los padres y en caso de ser más "apto" lo substituirá en la siguiente generación.

### Parametros del AG

In [ ]:
generaciones = 500
nR = 20
tamPob=100
probaMut=0.2
probaCruza=0.5

### Elitismo
Es el proceso de mantener siempre al mejor individuo encontrado durante la ejecución.

### Ejemplo completo

In [ ]:
# inicializa población y evalua fitness

stats={'min':[],'max':[],'media':[]}

poblacion=poblacionInicial(tamPob, nR)
fitness=list(map(aptitud,poblacion))

# Guardar valores para gráficas
stats['min'].append(np.min(fitness))
stats['max'].append(np.max(fitness))
stats['media'].append(np.mean(fitness))

# Inicia ciclo principal
for g in range(generaciones):
    for m in range(int(probaCruza*tamPob)):
        
        # selección de padres
        i=random.randint(0,tamPob -1)
        j=random.randint(0,tamPob -1)
        while i == j:
            j=random.randint(0,tamPob -1)
        
        # Cruza, mutacion y selección por torneo
        h=cruza(poblacion[i],poblacion[j])
        h=mutacion(h,probaMut)
        hf=aptitud(h)
        
        # selección
        p=random.choice([i,j])
        if hf <= fitness[p]:
            poblacion[p]=h
            fitness[p]=hf
                
    # Guardar valores para gráficas
    stats['min'].append(np.min(fitness))
    stats['max'].append(np.max(fitness))
    stats['media'].append(np.mean(fitness))
    
# Se busca al mejor individuo de la población
eindex=fitness.index(min(fitness))
elite = poblacion[eindex]
efit = fitness[eindex]
print(efit)

Presentandolo bonito pa' los amigos de PyMX

In [ ]:
dibujar_tablero(elite)

In [ ]:
dibujar_estadisticas(stats)

## Biblioteca geneticalgorithm2

Hay varias bibliotecas que implementan algoritmos genéticos, entre ellas `geneticalgorithm2` es mantenida activamente, es fácil de usar y tiene una variedad de opciones para cruza, mutación, elitismo, etc. Usaremos un problema sencillo como ejemplo: 

$$ 4\sin(x\pi) +6\sin(y\pi) + (x-1)^2 + (y-1)^2 $$ 

Primero calculamos el valor óptimo que minimice la función usando la técnica de BFGS, esto sólo con el fin de conocer el mismo y compararlo con los resultados del algoritmo genético. 

In [ ]:
x_ini = optimize.brute(f, (slice(-3,5,0.5),slice(-3,5,0.5)),finish=None) # **
x_opt = optimize.fmin_bfgs(f,x_ini)
x_opt

### Ejemplo con `geneticalgorithm2`

In [ ]:
from geneticalgorithm2 import geneticalgorithm2 as ga
from geneticalgorithm2 import Crossover, Mutations, Selection 
from geneticalgorithm2 import Population_initializer

var_bound =np.array([[-3,5]]*2)

model = ga(f2, dimension = 2, 
                variable_type='real', 
                 variable_boundaries = var_bound,
                 variable_type_mixed = None, 
                 function_timeout = 10,
                 algorithm_parameters={'max_num_iteration': None,
                                       'population_size':100,
                                       'mutation_probability':0.1,
                                       'elit_ratio': 0.01,
                                       'crossover_probability': 0.5,
                                       'parents_portion': 0.3,
                                       'crossover_type':'uniform',
                                       'mutation_type': 'uniform_by_center',
                                       'selection_type': 'roulette',
                                       'max_iteration_without_improv':None}
            )

model.run(
    no_plot = False, 
    disable_progress_bar = False,
    set_function = None, 
    apply_function_to_parents = False, 
    start_generation = {'variables':None, 'scores': None},
    studEA = False,
    population_initializer = Population_initializer(select_best_of = 1, local_optimization_step = 'never', local_optimizer = None),
    seed = None
    )

# Gracias :)
![hipercubo de 10 dim](./images/10cube.png)